In [4]:
import numpy as np

class AntColony:
    def __init__(self, distances, n_ants=25, max_iter=10, alpha=1, beta=2, rho=0.1, q=1, pheromone=1):
        self.distances = distances
        self.n_ants = n_ants
        self.max_iter = max_iter
        self.alpha = alpha
        self.beta = beta
        self.rho = rho
        self.q = q
        self.pheromone = pheromone
        self.n_cities = len(distances)
        self.pheromone_matrix = np.ones(self.distances.shape) / self.n_cities
        self.best_distance = float('inf')
        self.best_solution = []

    def run(self):
        for _ in range(self.max_iter):
            solutions = self.generate_solutions()
            self.update_pheromone(solutions)
            best_solution_index = np.argmin([self.distance(solution) for solution in solutions])
            if self.distance(solutions[best_solution_index]) < self.best_distance:
                self.best_distance = self.distance(solutions[best_solution_index])
                self.best_solution = solutions[best_solution_index]

    def generate_solutions(self):
        solutions = []
        for _ in range(self.n_ants):
            visited = [0]
            while len(visited) < self.n_cities:
                probabilities = self.probabilities(visited)
                next_city = self.select_next_city(probabilities, visited)
                visited.append(next_city)
            visited.append(0)  # Return to the starting city
            solutions.append(visited)
        return solutions

    def probabilities(self, visited):
        current_city = visited[-1]
        unvisited_cities = [city for city in range(self.n_cities) if city not in visited]
        probabilities = []
        total = 0
        for city in unvisited_cities:
            pheromone = self.pheromone_matrix[current_city][city]
            distance = self.distances[current_city][city]
            probabilities.append((pheromone ** self.alpha) * ((1 / distance) ** self.beta))
            total += (pheromone ** self.alpha) * ((1 / distance) ** self.beta)
        probabilities = [prob / total for prob in probabilities]
        return probabilities

    def select_next_city(self, probabilities, visited):
        return np.random.choice([city for city in range(self.n_cities) if city not in visited], p=probabilities)

    def update_pheromone(self, solutions):
        self.pheromone_matrix *= (1 - self.rho)
        for solution in solutions:
            distance = self.distance(solution)
            for i in range(len(solution) - 1):
                city1, city2 = solution[i], solution[i + 1]
                self.pheromone_matrix[city1][city2] += self.q / distance

    def distance(self, solution):
        total_distance = 0
        for i in range(len(solution) - 1):
            city1, city2 = solution[i], solution[i + 1]
            total_distance += self.distances[city1][city2]
        return total_distance   

# Example usage
if __name__ == "__main__":
    # Example distances matrix (replace with your actual distances matrix)
    distances = np.array([
        [0, 10, 15, 20],
        [10, 0, 35, 25],
        [15, 35, 0, 30],
        [20, 25, 30, 0]
    ])

    # Initialize and run the Ant Colony Optimization algorithm
    ant_colony = AntColony(distances)
    ant_colony.run()

    # Print the best solution found
    print("Best solution:", ant_colony.best_solution)
    print("Best distance:", ant_colony.best_distance)


Best solution: [0, 2, 3, 1, 0]
Best distance: 80
